In [1]:
import websocket
import requests
import json
import pandas as pd
import datetime as dt
import numpy as np
import time
import ta
from dotenv import load_dotenv
import os
from binance.client import Client
from IPython.display import clear_output
import winsound

In [2]:
frequency = 250  # Set Frequency To 2500 Hertz
duration = 300  # Set Duration To 1000 ms == 1 second
winsound.Beep(frequency, duration)

In [3]:
load_dotenv()

api_key = os.getenv('API_KEY')
api_secret = os.getenv('SECRET_KEY')

client = Client(api_key, api_secret, testnet = False)

spot = client.get_account()
s_bal = pd.DataFrame(spot['balances'])
print(s_bal)


    asset        free      locked
0     BTC  0.00000000  0.00000000
1     LTC  0.00000000  0.00000000
2     ETH  0.00000000  0.00000000
3     NEO  0.00000000  0.00000000
4     BNB  0.24622450  0.00000000
..    ...         ...         ...
527   RPL  0.00000000  0.00000000
528   GFT  0.00000000  0.00000000
529   GNS  0.00000000  0.00000000
530   SYN  0.00000000  0.00000000
531  LQTY  0.00000000  0.00000000

[532 rows x 3 columns]


In [4]:
timezone = 8
endpoint = 'wss://stream.binance.com:9443/ws'
symbol = 'ethusdt'
symbol_C = symbol.upper()
interval = '15m'

# start epoch till now
start_time = 1676681007000
end_time = round(time.time() * 1000)

# step between timestamps in milliseconds, 60000 = 1min 
step = 60000 * 3600

In [5]:
data = json.dumps({'method':'SUBSCRIBE','params':[symbol + '@kline_' + interval],'id':1})

In [6]:
# 先抓歷史資料好讓技術指標能成型
def get_historical(symbol, interval, start_time, end_time, step):
    
    rawdf = pd.DataFrame()
    
    url = "https://api.binance.com/api/v3/klines"
    
    for timestamp in range(start_time, end_time, step):
        params = {"symbol": symbol_C,
                  "interval": interval,
                  "startTime": timestamp,
                  "endTime": timestamp + step}
        response = requests.get(url, params=params).json()
        out = pd.DataFrame(response, columns = ["Open time", "Open", "High", "Low", "Close",
                                               "Volume", "Close_Time", "Quote asset volume",
                                               "Number of trades", "Taker buy base asset volume",
                                               "Taker buy quote asset volume", "Ignore"])
        rawdf = pd.concat([rawdf, out], axis = 0)
    
    rawdf = rawdf[['Close_Time', 'Open', 'Close', "High", "Low", 'Volume']]
    convert_dict = {'Close_Time': float, 'Open': float, 'Close': float, "High": float, "Low": float, 'Volume': float}
    rawdf = rawdf.astype(convert_dict)

    rawdf['Close_Time'] = pd.to_datetime(rawdf['Close_Time'], unit = 'ms')
    rawdf['Close_Time'] = rawdf['Close_Time'] + pd.Timedelta(hours=timezone)
    rawdf['Close_Time'] = rawdf['Close_Time'].dt.strftime('%Y-%m-%d %H:%M:%S')
    
    rawdf = rawdf.reset_index(drop=True)
    
    return rawdf

In [7]:
rawdf = get_historical(symbol, interval, start_time, end_time, step)
print(rawdf)

               Close_Time     Open    Close     High      Low     Volume
0     2023-02-18 08:59:59  1693.29  1694.88  1697.50  1692.46  3680.8808
1     2023-02-18 09:14:59  1694.89  1696.57  1697.45  1692.79  5764.4068
2     2023-02-18 09:29:59  1696.58  1698.31  1698.35  1694.30  2265.6904
3     2023-02-18 09:44:59  1698.30  1697.43  1699.66  1694.22  3123.9605
4     2023-02-18 09:59:59  1697.43  1702.51  1704.46  1697.42  4325.5238
...                   ...      ...      ...      ...      ...        ...
2792  2023-03-19 10:59:59  1779.01  1777.78  1780.06  1776.00  2015.5832
2793  2023-03-19 11:14:59  1777.78  1779.43  1779.89  1776.69  1500.6566
2794  2023-03-19 11:29:59  1779.42  1775.63  1781.69  1775.55  2288.9386
2795  2023-03-19 11:44:59  1775.62  1778.31  1778.42  1774.58  2094.3123
2796  2023-03-19 11:59:59  1778.31  1782.16  1783.88  1777.91  3047.0460

[2797 rows x 6 columns]


In [8]:
def indicators(df):
#     bband
    bb_int = 30
    bb_dev = 2
    bb = ta.volatility.BollingerBands(df['Close'], window=bb_int, window_dev=bb_dev)
    df['bb_u'] = bb.bollinger_hband()
    df['bb_m'] = bb.bollinger_mavg()
    df['bb_l'] = bb.bollinger_lband()
#     rsi
    rsi_int = 14
    df['rsi'] = ta.momentum.RSIIndicator(df['Close'], window = rsi_int).rsi()
# kd
    kd_int = 14
    d_int = 3
   
    kddf = pd.DataFrame()
    kddf[str(kd_int) + '-Low'] = df['Low'].rolling(kd_int).min()
    kddf[str(kd_int) + '-High'] = df['High'].rolling(kd_int).max()
    df['slowk'] = (df['Close'] - kddf[str(kd_int) + '-Low'])*100/(kddf[str(kd_int) + '-High'] - kddf[str(kd_int) + '-Low'])
    df['slowd'] = df['slowk'].rolling(d_int).mean()
    return df

In [9]:
def conditions(df):

    # c1 下影線 大於 實體Ｋ & 下影線 大於 上影線
    # c2 收盤 小於 布林下
    df['c1_1'] = (abs(df['Low'] - df['Close']) > abs(df['Open'] - df['Close'])) & (abs(df['Low'] - df['Close']) > abs(df['High'] - df['Open']))
    df['c1_2'] = (abs(df['Low'] - df['Open']) > abs(df['Open'] - df['Close'])) & (abs(df['Low'] - df['Open']) > abs(df['High'] - df['Close']))
    df['c2'] = df['Close'] < df['bb_l']

    # 條件達成
#     if df.iloc[-1].c1_1 & df.iloc[-1].c1_2 & df.iloc[-1].c2 == True
    df['signal'] = False
    df.loc[df.c1_1 & df.c1_2 & df.c2 , 'signal'] = True
            
    # 下一根進場
    df['openbuy'] = False
#     df.loc[df['signal'].shift() == True, 'openbuy'] = True
    return df

In [10]:
def enter_position(df):
    global symbol_C
    
    #-----Calculate entry price-----#
    close_val = df['Close']
    low_val = df['Low']
    df.loc[df.index[-1], 'entry_p'] = close_val.loc[close_val.index[-2]]
    df.loc[df.index[-1], 'stop_loss'] = low_val.loc[low_val.index[-2]]

    #-----position attributes-----#
    quantity = 0.025  # the amount of the cryptocurrency you want to buy or sell
    entry_p = df.loc[df.index[-2], 'Close']  # the price at which you want to buy or sell
    stop_loss_p = df.loc[df.index[-2], 'Low'] * 0.999  # the stop loss price
#     stop_limit_price = df.loc[df.index[-1], 'stop_loss']  # the limit price of the stop loss order
#     stop_loss_quantity = 0.005  # the amount of the cryptocurrency you want to sell in the stop loss order

    print(df.loc[df.index[-1], 'entry_p'] == entry_p)  
    
    try:
        order = client.futures_create_order(
            symbol=symbol_C,
            side='BUY',
            type='LIMIT',
            timeInForce='GTC',
            quantity=quantity,
            price=entry_p,
            positionSide='LONG'
        )
        print('Order created successfully.')       
        
        stop_loss_order = client.futures_create_order(
            symbol=symbol_C,
            side='SELL',
            type='STOP_MARKET',
            stopPrice=stop_loss_p,
            stopLimitTimeInForce='GTC',
            closePosition = 'true',
            positionSide = 'LONG'
#             quantity=stop_loss_quantity,
#             stopLimitPrice=stop_limit_price,
        )
        print('Stop loss created successfully')
            
    except Exception as e:
        print(f'Error creating order: {e}')

    print(str(symbol_C) + ' entered at ' + str(entry_p) + ', stop loss at ' + str(stop_loss_p))


In [11]:
line_count = 0
max_lines = 5

current_k = 0
    
tp_rsi = 68
tp_slowk = 80
tp_slowd = 80

def on_open(ws):
    ws.send(data)
    
def on_message(ws, message):
    global current_k, tp_rsi, tp_slowk, tp_slowd, line_count, max_lines
    df = get_historical(symbol, interval, start_time, end_time, step)

    indicators(df)
    conditions(df)
    
    try:
        # 如果時間標籤改變，代表上個 Close 發生，檢查是否進場
        if df['Close_Time'][len(df) - 1] != current_k:
            print('time changed')
            current_k = df['Close_Time'][len(df) - 1]
            if df.iloc[df.index[-2]]['signal'] == True:
                winsound.Beep(frequency, duration)
                df.loc[df.index[-1], 'openbuy'] = True
                enter_position(df)
                print(df.tail(2))
        
        # 隨時偵測是否有部位。出場條件是否成立，不必等 Close 發生
        positions = client.futures_account()['positions']
        for position in positions:
            if position['symbol'] == symbol_C and float(position['positionAmt']) != 0:
                if (df.loc[df.index[-1], 'Close'] >= df.loc[df.index[-1], 'bb_u'] * 0.9998):
                    take_profit_order = client.futures_create_order(
                        symbol=symbol_C,
                        side='SELL',
                        type='TAKE_PROFIT_MARKET',
                        stopPrice=df.loc[df.index[-1], 'Close'],
                        stopLimitTimeInForce='GTC',
                        closePosition = 'true',
                        positionSide = 'LONG'
                    )
                    winsound.Beep(frequency, duration)
                    print('Take profit created successfully')
        
    except Exception as e:
        print(f'Error creating order: {e}')


    try:
        df = df.reset_index(drop=True)
        print('nothing happened...' + str(df.loc[df.index[-1], 'Close_Time']) + ' ' + str(df.loc[df.index[-1], 'Close']))
        print()
        print(df.tail(1))
        print('----------------------------------------------------------------------------')
        
        positions = client.futures_account()['positions']
        for position in positions:
            if float(position['positionAmt']) != 0:
                position_df = pd.DataFrame({'Symbol':position['symbol'],
                                            'Entry_P':round(float(position['entryPrice']),2),
                                            'Amt':round(float(position['positionAmt']) * df.loc[df.index[-1], 'Close'],2),
                                            'PL':round(float(position['unRealizedProfit']),2),
                                            'X':round(float(position['leverage']),1),
                                            'Liquid_P':round(float(position['liquidationPrice']),2)
                                           }, index=[0])      
                print(position_df)
    
    except Exception as e:
        print(f'Error: {e}')    
        
        
    line_count += 1
    if line_count >= max_lines:
        clear_output(wait=True)
        line_count = 0

In [ ]:
ws = websocket.WebSocketApp(endpoint, on_message = on_message, on_open = on_open)
ws.run_forever()

nothing happened...2023-03-19 11:59:59 1783.02

               Close_Time     Open    Close     High      Low     Volume  \
2796  2023-03-19 11:59:59  1778.31  1783.02  1783.88  1777.91  3388.4474   

             bb_u      bb_m         bb_l      rsi      slowk      slowd  \
2796  1794.945066  1776.254  1757.562934  55.0859  81.583578  68.827647   

       c1_1   c1_2     c2  signal  openbuy  
2796  False  False  False   False    False  
----------------------------------------------------------------------------
nothing happened...2023-03-19 11:59:59 1782.69

               Close_Time     Open    Close     High      Low     Volume  \
2796  2023-03-19 11:59:59  1778.31  1782.69  1783.88  1777.91  3416.0052   

             bb_u      bb_m         bb_l        rsi      slowk      slowd  \
2796  1794.918507  1776.243  1757.567493  54.783935  79.648094  68.182485   

       c1_1   c1_2     c2  signal  openbuy  
2796  False  False  False   False    False  
-----------------------------------